<img src="Images/HSP2.png" />
This Jupyter Notebook Copyright 2017 by RESPEC, INC.  All rights reserved.

$\textbf{HSP}^{\textbf{2}}\ \text{and}\ \textbf{HSP2}\ $ Copyright 2017 by RESPEC INC. and released under this [License](LegalInformation/License.txt)

# TUTORIAL 5: $\textbf{HSP}^\textbf{2}$  Plotting and Reporting

## Introduction

This tutorial will cover how to make publication quality plots and create reports from the HDF5 simulation data.

**Tutorial Contents**

 + Section 1: [Plotting](#section1)
 + Section 2: [Reports](#section2)
 + Section 3: [Using Pandas for Reports](#section3)

### Required Python imports

In [ ]:
import os
import site
site.addsitedir(os.getcwd().rsplit('\\',1)[0] + '\\')  # adds your path to the HSP2 software.

hdfname = os.path.join('TutorialData', 'Tutorial.h5')

import shutil
import numpy as np
import pandas as pd
pd.options.display.max_rows    = 18
pd.options.display.max_columns = 10
pd.options.display.float_format = '{:.2f}'.format  # display 2 digits after the decimal point

from matplotlib import pyplot as plt
%matplotlib inline

import HSP2
import HSP2tools

HSP2tools.reset_tutorial()    # make a new copy of the tutorial's data
HSP2tools.versions()          # display version information below

## Section 1: Plotting<a id='section1'></a>

It is easy to make professional quality plots. Pandas uses the matplotlib  library under the hood which provides essentially all the plot
options in MATLAB (2-d only).

The calculated VOLEV timeseries from Reach 1 and Reach 5 will be used to demonstrate the concepts.

**NOTE** When a user specifies saving a calculated timeseries (this will be shown later), the entire timeseries is saved at the timestep of the simulation. This allows resampling at convenient timesteps (like days, weeks, months, years) for both plotting and creating written reports.  Timeseries that are needed for inputs to other modules are automatically saved. The remaining timeseries are only saved when specified by the user.

In [ ]:
HSP2.run(hdfname, saveall=True)  # make sure the computed results are available

#### Read the VOLEV data from the tutorial HDF5 file for segments '1' and '5'

In [ ]:
df1 = pd.read_hdf(hdfname, '/RESULTS/RCHRES_R001/HYDR')['VOLEV']
df5 = pd.read_hdf(hdfname, '/RESULTS/RCHRES_R005/HYDR')['VOLEV']

Display the first few values to see that the timeseries is as expected and is hourly (Freq is H for hourly).

**NOTE** the function **head()** used below is just a simple method to display the first few lines of data. The function **tail()** similarly shows the last few data items.  Either function can take an integer argument to specify the number of lines to display.

In [ ]:
df1.head()

#### Make a simple plot.

In [ ]:
df1.plot(figsize=[20, 10])

This is a pretty messy plot, but works easily.

Now aggregate the data into daily time intervals and select the evaporation sum for each day for a cleaner plot.

**Note** The aggregation method is "sum" which adds up all the values falling into the new time interval.

In [ ]:
df1_dailysum = df1.resample('D', how='sum')
df1_dailysum.head(10)

In [ ]:
df1_dailysum.plot(figsize=[20, 10])

#### Maybe a weekly aggregation is less messy, but still shows the data well.

In [ ]:
df1_weeklysum = df1.resample('W', how='sum')
df1_weeklysum

In [ ]:
df1_weeklysum.plot(figsize=[20, 10])

#### OK, better. Now do this for the REACH 5 timeseries.

In [ ]:
df5_weeklysum = df5.resample('W', how='sum')
df5_weeklysum

#### Make a plot using both timeseries with labels and legend

In [ ]:
df1_weeklysum.plot(style='red',  label='REACH1', figsize=[20, 10]) 
df5_weeklysum.plot(style='blue', label='REACH5')
plt.title('Weekly Sum of Evaporation, VOLEV')    # can set titles
plt.ylabel('acre-ft/ hour')                      # can set axis labels
plt.legend(loc='best')                           # make a legend

# demonstrate saving the plots to files in various format
plt.savefig('tutorial5.png')
plt.savefig('tutorial5.pdf')
plt.savefig('tutorial5.tiff')
plt.savefig('tutorial5.svg')

Pandas plot() makes prettier time axis than matplotlib directly, but you can use matplotlib routines for other the options like legends and titles.

The matplotlib library generally has every option that MATLAB has for 2-d plotting with similar names and arguments. This makes it easy to generate publication quality plots.

You can open the PNG, PDF, TIFF, and SVG files in the TutorialsData directory and view them with your viewing tools.

### Some background on interactive plotting

There is very active development going on to visualize "big" data (including plotting) interactively. This is being driven by big science and also by "Quants", Quantitative Analysts and Data Scientists (new fields) seeking to process a lot of data fast and visualize it quickly in order to make stock market or trading decisions.
These development projects all rely  on advanced HTML 5 features. Different browsers have different levels of compliance to HTML 5, so sometimes some tool features break on any specific browser.

Some links to a very few of the many interesting projects:
    
 + [D3](http://d3js.org/) This is the great grand daddy of these efforts. the weird looking display of hexagons are selected examples. Click on one and see it demonstrated.
 + [Ipython Vega](https://github.com/vega/ipython-vega) This project is under active development to bring Vega to the Jupyter Notebook.
 + [plotly](https://plot.ly/). This was a commercial product - but recently became free to use.
 + [bokeh](http://bokeh.pydata.org/) Continuum Analytics won a large (3 million dollar) grant from DARPA's XDATA project. It requires all the funded projects to become open source. It is D3 like in look & feel, but very different visualization grammar. Click on one of the images to see it work and to see the code. (Most seem to be interactive examples, but not all.) Well integrated with IPython Notebook.
 + [mpld3](http://mpld3.github.io/)  This project which converts matplotlib calls to D3. 

#### Using Matplotlib to make interactive plots

A feature of the 1.4 matplotlib is the new notebook backend to make it easy to make interactive plots.
Put this in your Python imports and settings cell:

> %matplotlib notebook

instead of 

> %matplotlib inline

Note the tools to pan, zoom, and reset the plot. Hover over a tool icon and a hint will appear to describe that tool. You can zoom into specific hourly data and pan smoothly to any part of the timeseries. Hover your mouse over the plotted data and the time and value are displayed below the plot frame.

When you are done, click on the x in the red box at the upper right corner of the plot which freezes the plot in the current view and exits from the interactive mode in that cell. 

Here is an example using an hourly precpitation time series.

**Note** the Jupyter Notebook is not really good at allowing changes to the matplotlib backend once the Notebook is started. 

Currently, you should not mix backends. But for this tutorial as the last plot, we will try this.

(It might fail. In this case, you need to remove the inline statement near the top of this notebook and replace it with the matplotlib notebook.  Save the notebook and restart the kernel. Then it should work.  Active development is being made to embed the plotting package into the Jupyer Notebook in a way to allow more dynamic changes of the backend.)

In [ ]:
%matplotlib notebook
df1 = pd.read_hdf(hdfname, 'TIMESERIES/TS39')
df1.plot()

**Section Summary**

 + Demonstrated disaggregation of the data from hourly to daily and weekly intervals
 + Demonstrated making plots with
   + multiple timeseries
   + plot Title
   + plot Axis Labels
   + control of line styles (color)
   + Plot legend
 + Demonstrated interactive plotting
 + Demostrated saving plots in a variety of formats
   + SVN
   + PNG
   + PDF
   + TIFF

## Section 2: Reports<a id='section2'></a>

Most of the computed time series datasets are needed for this example. Use the saveall option to insure the necessary data are available.

In [ ]:
HSP2.run(hdfname, saveall=True)

### Demonstrate creating report like traditional HSPF

The following is part of HSPF test 10 output.  This tutorial will show how to make a similar report - but covering all months at once.

### PERLND SNOW

In [ ]:
snowpath = '/RESULTS/PERLND_P001/SNOW'

#### Snow - State Variables

The following defines the state variables in the order they should occur in the report.

In [ ]:
snow_state_variables = ['PACKF', 'PACKW', 'PACKI','PDEPTH','COVINX', 'NEGHTS', 'XLNMLT', 'RDENPF',
 'SKYCLR', 'SNOCOV',  'DULL', 'ALBEDO', 'PAKTMP', 'DEWTMP', 'SNOTMP']

The following reads the entire snow data from the HDF5 file, then selects only the state variables defined above

In [ ]:
snowsv = pd.read_hdf(hdfname, snowpath)[snow_state_variables]
snowsv.head()

Now convert the frequency to "monthly" 

In [ ]:
snowsv = snowsv.resample('M', how='last')
snowsv

You can see the HSP2 answers for the same simulation test in the table above including the same number of digits.
They following shows how this might print.  The first column could be "prettyed up" to show just the month or month/year as desired.

In [ ]:
print snowsv

You can print directly to a file almost as easily.

In [ ]:
with open('TutorialData/print_example.txt', 'w') as f:
    print >>f, snowsv

Open this file with any text editor or view in your [browser](TutorialData/print_example.txt)

#### SNOW - Fluxes

In [ ]:
snow_fluxes = ['SNOWF', 'PRAIN', 'SNOWE',  'WYIELD', 'MELT', 'RAINF']

In [ ]:
snow_flx = pd.read_hdf(hdfname, snowpath)[snow_fluxes]
snow_flx

Again convert to Monthly frequency

In [ ]:
snow_flx = snow_flx.resample('M', how='sum')
snow_flx

### PERLND WATER

#### WATER - State Variables

In [ ]:
waterpath = '/RESULTS/PERLND_P001/PWATER'

In [ ]:
water_state_variables = ['PERS', 'CEPS', 'SURS', 'UZS', 'IFWS', 'LZS', 'AGWS', 'GWVS', 'INFFAC', 'PETADJ', 'TGWS']
watersv = pd.read_hdf(hdfname, waterpath)[water_state_variables].resample('M', how='last')
watersv

#### Water - Fluxes (External)

In [ ]:
pd.options.display.float_format = '{:.3f}'.format  # display 3 digits after the decimal point

water_fluxes_external = ['SUPY', 'SURO', 'IFWO', 'AGWO',  'PERO','IGWI', 'SURI']
water_flx_external = pd.read_hdf(hdfname, waterpath)[water_fluxes_external].resample('M', how='sum')
water_flx_external

#### Water - Fluxes  (Evapotranspiration)

In [ ]:
water_fluxes_ev = ['CEPE', 'UZET',  'LZET', 'AGWET', 'BASET', 'TAET']
water_flx_ev = pd.read_hdf(hdfname, waterpath)[water_fluxes_ev].resample('M', how='sum')
water_flx_ev

#### Water - Fluxes  (Internal Fluxes)

In [ ]:
water_fluxes_internal =  ['IFWI', 'UZI',  'INFIL', 'PERC',  'LZI', 'AGWI', 'SURI']
water_flx_internal = pd.read_hdf(hdfname, waterpath)[water_fluxes_internal].resample('M', how='sum')
water_flx_internal

#### Fetch name of this segment from the HDF5 file.

In [ ]:
segment = 'P001'
name = pd.read_hdf(hdfname, '/PERLND/GENERAL_INFO').loc[segment, 'LSID']
name

### Now put this together into one report.

Change display options for printed page rather than this Notebook display.

In [ ]:
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 150)
pd.set_option('display.show_dimensions', False)

In [ ]:
with open('TutorialData/print_example2.txt', 'w') as f:
    print >>f, 'PERVIOUS Land Segment', segment, name
    print >>f
    print >>f, 'SNOW State Variables'
    print >>f
    print >>f, snowsv
    print >>f
    print >>f, 'SNOW Fluxes'
    print >>f
    print >>f, snow_flx
    print >>f
    print >>f, 'PWATER State Variables'
    print >>f
    print >>f, watersv
    print >>f
    print >>f, 'PWATER FLUXES'
    print >>f
    print >>f, 'External Inflows and Outflows'
    print >>f
    print >>f, water_flx_external
    print >>f
    print >>f, 'Evapotranspiration Potential'
    print >>f
    print >>f, water_flx_ev
    print >>f
    print >>f, 'Internal Fluxes'
    print >>f
    print >>f, water_flx_internal

Open the report file, print_example2.txt, with any text editor or view it in your [browser](TutorialData/print_example2.txt)

Additional formatting or content can customize the report as necessary.

The user can create Python files for specific report needs (including plots) and run them in one step.

## Section 3: Using Pandas for Reports<a id='section3'></a>

Pandas is useful in "munging" data into the required form for reports. These tutorials have demonstrated converting timeseries data from hourly data into annual, monthly, and weekly intervals with various aggregation methods including max, sum, and last.

The following image shows the basic intervals available for reports. Multiples of these intervals are also available. For example, 'H' is an hourly interval and '4H' is every4 hours.  Pandas takes care of leap years, time zones, and other similar issues.

<img src="files/Images/FrequencyConversion.png" />

Frequently reports need intervals based on different starting dates. For example, to create an annual report from June to June.
These are called Anchored Offsets.  For example, 'A-OCT' is an annual frequence starting and ending at the end of October (October 31).

<img src="files/Images/AnchoredOffsets.png" />

There are many options for aggregating/disagregating the data. For example, these are the available interpolation methods:

 + linear
 + time
 + values
 + index
 + nearest
 + zero
 + slinear
 + quadratic
 + cubic
 + barycentric
 + krogh
 + polynomial
 + spline
 + piecewise_polynomial
 + pchip
    

The "how" function in the resample() can be any  numpy array function or user defined array function including

 + abs
 + max
 + min
 + mean (default)
 + std
 + last
 + pad (forward and backward)
 + sum, nansum
 + prod
 + cumsum
 + cumprod
 + diff
 
Users can specify which endpoint (right or left) is included in the intervals

Users can use many database operations between Pandas tables including

 + merges (inner, outer, left, right)
 + groupby
 + concat, join, append
 + pivot
 + multiindex
 


## Other Reporting Options

The IPython Notebook has a tool **nbconvert** that can convert a notebook into HTML, $\LaTeX$, PDF, and slide show (jsRevel) formats. Look under the "File" menu at the top of the page. Select it and then select "Download as" to see some options.  In order to convert to some formats, additional software needs to be installed. For example, to convert to $\LaTeX$ or PDF, $\TeX$ installation (such as MikTex) must be installed. You can try converting this notebook to HTML (since everything is available) to try this feature.


The nbcovert tool
can also be run from a command prompt window with more options. It is a bit brittle in creating LaTeX and PDF files, but
works. The PDF is created from an intermediate LaTex representation - so it looks really good. The documentation is at [nbconvert](http://ipython.org/ipython-doc/stable/notebook/nbconvert.html). Pandas and Bokeh are improving the **repr** representation of their
objects(plots, timeseries, tables, etc.) for better display/printing in HTML and LaTex. 

View the PDF files created from Tutorial 1 and Tutorial 3 to see the results of nbconvert. (These files are located in the ExampleData directory.)  A few items are either missing or are a bit messed up, but the overall documents only need a bit of tweaking for release. The corresponding $\TeX$ files are not included. Assuming that the command window has the current directory set to the directory running these tutorials the command to create a PDF file is

    ipython nbconvert --to PDF "Tutorial 1.ipynb"
  
and to create a reveal slideshow (and start it running) is

    ipython nbconvert --to slides --post serve "Tutorial 1.ipynb"
  
The name of the file had to be in quotes because of the space in the name.

Due in part to the Sloan Foundation Grant and Microsoft Grant, the IPython team is rapidly improving the notebook capabilities to
generate documentation in many formats.  The [IPython Example Notebook View Page](http://nbviewer.ipython.org/) show the Notebook supporting 3 languages and
3 books written in the Notebook.  The [Gallery](https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks)
shows many more examples including a growing number of Acedemic publications.

New publishing concepts are rapidly developing. [Authorea](https://www.authorea.com/)  is a service to create technical articles which now
supports IPython Notebooks.

So hydrologists using Notebooks to run HSP2 can actually use their Notebooks to record their research and workflow, then generate documentation directly from these Notebooks. A great example of professional hydrology notebooks are by Dr. Jeffrey Kantor for the [Rainly Lake region](http://jckantor.github.io/Rainy-Lake-Hydrology/)